In [ ]:
pip install faker

In [ ]:
import pandas as pd
import datetime
import random
from faker import Faker

In [ ]:
fake = Faker()
Faker.seed(42)
random.seed(42)

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/stepthom/869_course/main/data/GermanCredit.csv")
df.info()

In [ ]:
today = datetime.date(2021, 6, 1)
print(today)

In [ ]:
dob = today - datetime.timedelta(days=40*365)
print(dob)

In [ ]:
df.head()

In [ ]:
#Generate 20 random cities:
n = 20
cities = [fake.city() for _ in range(20)]
print(cities)
cities_probs = [1.0/(i+1) for i in range(20)]
cities_probs_norm =  [float(i)/sum(cities_probs) for i in cities_probs]
print(cities_probs_norm)


In [ ]:

for i in df.index:
    
    # Age
    days = df.loc[i, 'Age']*365 + (random.randrange(365*2) - 365)
    dob = today - datetime.timedelta(days=int(days))
    df.at[i, 'DateOfBirth'] = dob
    
    if df.loc[i, 'Personal.Male.Divorced.Seperated'] == 1 or df.loc[i, 'Personal.Male.Single'] == 1 or df.loc[i, 'Personal.Male.Married.Widowed'] == 1:
        df.at[i, 'Sex'] = "M"
        df.at[i, 'FirstName'] = fake.first_name_male()
    else:
        df.at[i, 'Sex'] = "F"
        df.at[i, 'FirstName'] = fake.first_name_female()
        
    df.at[i, 'LastName'] = fake.last_name()
        
    if df.loc[i, 'Personal.Male.Married.Widowed'] == 1 or df.loc[i, 'Personal.Female.NotSingle'] == 1:
        df.at[i, 'Married'] = int(1)
    else:
        df.at[i, 'Married'] = int(0)
    

    # Nonsense feature
    df.loc[i, 'NumberPets'] = random.randrange(3)
    
    if df.loc[i, 'EmploymentDuration.lt.1'] == 1:
        df.at[i, 'EmploymentDuration'] = 0
    elif df.loc[i, 'EmploymentDuration.1.to.4'] == 1:
        df.at[i, 'EmploymentDuration'] = 1 + random.randrange(4)
    elif df.loc[i, 'EmploymentDuration.4.to.7'] == 1:
        df.at[i, 'EmploymentDuration'] = 4 + random.randrange(4)
    elif df.loc[i, 'EmploymentDuration.gt.7'] == 1:
        df.at[i, 'EmploymentDuration'] = 7 + random.randrange(10)
        
    if df.loc[i, 'SavingsAccountBonds.Unknown'] == 1:
        df.at[i, 'SavingsAccountBonds'] = random.randrange(600)
    elif df.loc[i, 'SavingsAccountBonds.lt.100'] == 1:
        df.at[i, 'SavingsAccountBonds'] = 0
    elif df.loc[i, 'SavingsAccountBonds.100.to.500'] == 1:
        df.at[i, 'SavingsAccountBonds'] = 100 + random.randrange(401)
    elif df.loc[i, 'SavingsAccountBonds.500.to.1000'] == 1:
        df.at[i, 'SavingsAccountBonds'] = 500 + random.randrange(501)
    elif df.loc[i, 'SavingsAccountBonds.gt.1000'] == 1:
        df.at[i, 'SavingsAccountBonds'] = 1000 + random.randrange(1000)
        
        
    inflation = 1.8
    
    df.at[i, 'Amount'] = round(df.loc[i, 'Amount'] * inflation)
    
   
    df.at[i, 'Street'] = fake.street_address()
    #df.at[i, 'City'] = fake.city()
    
    df.at[i, "LicensePlate"] = fake.license_plate()
    
    df.at[i, 'UserID'] = fake.ssn()
    
df['City'] = random.choices(cities, cities_probs_norm, k=df.shape[0])

In [ ]:
df = df.drop(['Age'], axis=1)
df = df.rename(columns={'Telephone': 'OwnCar', 'Class': "Credit"})
df = df[df.columns.drop(list(df.filter(regex='Personal.')))]
df = df[df.columns.drop(list(df.filter(regex='EmploymentDuration.')))]
df = df[df.columns.drop(list(df.filter(regex='SavingsAccountBonds.')))]

In [ ]:
df['BadCredit'] = df['Credit'].apply(lambda x: 0 if x == 'Good' else 1)
df = df.drop('Credit', axis=1)

In [ ]:
front_cols = ['UserID', 'FirstName', 'LastName', 'DateOfBirth', 'Sex', 'Street', 'City', 'LicensePlate', 'Married', "NumberPets"]

df = df[ front_cols + [ col for col in df.columns if col not in front_cols] ]

last_cols = ['BadCredit']
df = df[ [ col for col in df.columns if col not in last_cols] + last_cols ]

In [ ]:
df.head()

In [ ]:
df['BadCredit'].value_counts()

In [ ]:
df.to_csv('generated_german.csv', index=False)